## Requirements:

In [ ]:
import gurobipy as gpfrom gurobipy import GRBimport jsonimport pandas as pdimport numpy as npfrom pathlib import Pathimport matplotlib.pyplot as plt

## Data:

In [ ]:
data_dir = Path("./")data_dir_appliance_params = data_dir / "appliance_params.json"data_dir_appliance_params = data_dir_appliance_params.resolve()with open(f"{data_dir_appliance_params}", "r") as f:    data = json.load(f)data

In [ ]:
data_dir = Path("./")data_dir_bus_params = data_dir / "bus_params.json"data_dir_bus_params = data_dir_bus_params.resolve()with open(f"{data_dir_bus_params}", "r") as f:    data2 = json.load(f)data2

In [ ]:
data_dir = Path("./")data_dir_DER_production = data_dir / "DER_production.json"data_dir_DER_production = data_dir_DER_production.resolve()with open(f"{data_dir_DER_production}", "r") as f:    data3 = json.load(f)data3

In [ ]:
data_dir = Path("./")data_dir_usage_preferences = data_dir / "usage_preferences.json"data_dir_usage_preferences = data_dir_usage_preferences.resolve()with open(f"{data_dir_usage_preferences}", "r") as f:    data4 = json.load(f)data4

In [ ]:
def get_param(data, key, category=None, idx=0, subcategory=None, subidx=0):    try:        if category and subcategory:            return data[category][idx][subcategory][subidx][key]        elif category:            return data[category][idx][key]        elif subcategory:            return data[idx][subcategory][subidx][key]        elif isinstance(data, list):            return data[idx][key]        else:            return data[key]    except (KeyError, IndexError, TypeError):        print(f"Could not find {key} in the provided path.")        return None

In [ ]:
max_power_solar = get_param(data, "max_power_kW", category="DER")min_power_ratio = get_param(data, "min_power_ratio", category="DER")max_ramp_up_rate_solar  = get_param(data, "max_ramp_rate_up_ratio", category="DER")max_ramp_down_rate_solar = get_param(data, "max_ramp_rate_down_ratio", category="DER")max_FFL = get_param(data, "max_load_kWh_per_hour", category="load")min_FFL = get_param(data, "min_load_ratio", category="load")max_ramp_up_rate_FFL = get_param(data, "max_ramp_rate_up_ratio", category="load")max_ramp_down_rate_FFL = get_param(data, "max_ramp_rate_down_ratio", category="load")min_on_time_h = get_param(data, "min_on_time_h", category="load")min_off_time_h = get_param(data, "min_off_time_h", category="load")storage_capacity = get_param(data, "storage_capacity_kWh", category="storage")max_charging_power_ratio = get_param(data, "max_charging_power_ratio", category="storage")max_discharging_power_ratio = get_param(data, "max_discharging_power_ratio", category="storage")charging_efficiency = get_param(data, "charging_efficiency", category="storage")discharging_efficiency = get_param(data, "discharging_efficiency", category="storage")

In [ ]:
price_import = get_param(data2, "import_tariff_DKK/kWh")  price_export = get_param(data2, "export_tariff_DKK/kWh")  max_import = get_param(data2, "max_import_kW")  max_export = get_param(data2, "max_export_kW")  penalty_excess_import = get_param(data2, "penalty_excess_import_DKK/kWh") penalty_excess_export = get_param(data2, "penalty_excess_export_DKK/kWh")  electricity_price = get_param(data2, "energy_price_DKK_per_kWh")  

In [ ]:
Solar_CF = get_param(data3,"hourly_profile_ratio")load_profile_consumer = get_param(data4, "hourly_profile_ratio", subcategory="load_preferences")SOC_initial = get_param(data4, "initial_soc_ratio", subcategory="storage_preferences")SOC_final = get_param(data4, "final_soc_ratio", subcategory="storage_preferences")

In [ ]:
t = range(len(Solar_CF))penalty_discomfort = electricity_pricefor _ in range(len(penalty_discomfort)):    penalty_discomfort[_] *= 1.7

## Optimization model:

In [ ]:
def prosumer_optimization(price_import, price_export, electricity_price,                          penalty_excess_import, penalty_excess_export):    model = gp.Model("Prosumer_Continuous")    x_imports = model.addVars(t, name="imported_power_kW", lb=0, ub=GRB.INFINITY)    x_exports = model.addVars(t, name="exported_power_kW", lb=0, ub=GRB.INFINITY)    x_FFL = model.addVars(t, name="FFL_consumption_kW", lb=0, ub=1)    x_solar = model.addVars(t, name="solar_power_kW", lb=0)    z_import_excess = model.addVars(t, name="excess_import_kW", lb=0)    z_export_excess = model.addVars(t, name="excess_export_kW", lb=0)    delta_FFL = model.addVars(t, name="delta_FFL", lb=0, ub=GRB.INFINITY)    SOC_t = model.addVars(t, name="SOC", lb=0, ub=storage_capacity)    x_charging = model.addVars(t, name="charging_power_kW", lb=0, ub=max_charging_power_ratio * storage_capacity)    x_discharging = model.addVars(t, name="discharging_power_kW", lb=0, ub=max_discharging_power_ratio * storage_capacity)    for i in t:        model.addConstr(x_FFL[i] * max_FFL >= min_FFL, name=f"FFL_min_{i}")        model.addConstr(x_FFL[i] * max_FFL <= max_FFL, name=f"FFL_max_{i}")        model.addConstr(x_solar[i] <= Solar_CF[i] * max_power_solar, name=f"solar_max_{i}")        model.addConstr(            x_imports[i] - x_exports[i] + x_solar[i] + x_discharging[i] - x_charging[i] == x_FFL[i] * max_FFL,            name=f"energy_balance_{i}"        )        model.addConstr(z_import_excess[i] >= x_imports[i] - max_import, name=f"excess_import_{i}")        model.addConstr(z_export_excess[i] >= x_exports[i] - max_export, name=f"excess_export_{i}")        model.addConstr(delta_FFL[i] >= (x_FFL[i] - load_profile_consumer[i]) * max_FFL, name=f"discomfort_pos_{i}")        model.addConstr(delta_FFL[i] >= (load_profile_consumer[i] - x_FFL[i]) * max_FFL, name=f"discomfort_neg_{i}")        if i == 0:            model.addConstr(SOC_t[i] == SOC_initial * storage_capacity, name="initial_soc")        if i == 23:            model.addConstr(SOC_t[i] == SOC_final * storage_capacity, name="final_soc")        if i > 0:            model.addConstr(                SOC_t[i] == SOC_t[i-1] + (x_charging[i - 1] * charging_efficiency) - (x_discharging[i - 1] / discharging_efficiency),                name=f"SOC_dynamics_{i}"            )    model.setObjective(    gp.quicksum(        (-price_import - electricity_price[i]) * x_imports[i]        - penalty_excess_import * z_import_excess[i]        + (electricity_price[i] - price_export) * x_exports[i]        -penalty_excess_export * z_export_excess[i] - penalty_discomfort[i]*delta_FFL[i]        for i in t    ),    sense=GRB.MAXIMIZE)    model.optimize()    return model, x_imports, x_exports, x_FFL, x_solar, delta_FFL, SOC_t, x_charging, x_discharging

In [ ]:
model, x_imports, x_exports, x_FFL, x_solar, delta_FFL, SOC_t, x_charging, x_discharging = prosumer_optimization(    price_import, price_export, electricity_price,    penalty_excess_import, penalty_excess_export)print("Hour | Import(kW) | Export(kW) | FFL(kW) | Solar(kW) | Delta_FFL | SOC(kWh) | Charging(kW) | Discharging(kW)")for i in t:    print(f"{i:>4} | "          f"{x_imports[i].X:>10.3f} | "          f"{x_exports[i].X:>10.3f} | "          f"{x_FFL[i].X * max_FFL:>7.3f} | "          f"{x_solar[i].X:>8.3f} | "          f"{delta_FFL[i].X:>9.3f} | "          f"{SOC_t[i].X:>8.3f} | "          f"{x_charging[i].X:>11.3f} | "          f"{x_discharging[i].X:>13.3f}")print("Optimal objective value:", model.ObjVal)

In [ ]:
import matplotlib.pyplot as plthours = list(t)soc = [SOC_t[i].X for i in hours]imports = [x_imports[i].X for i in hours]exports = [x_exports[i].X for i in hours]discomfort = [delta_FFL[i].X for i in hours]actual_profile = [x_FFL[i].X * max_FFL for i in hours]solar_production = [x_solar[i].X for i in hours]plt.figure(figsize=(12, 7))plt.plot(hours, soc, label="SOC (kWh)")plt.plot(hours, imports, label="Import (kW)")plt.plot(hours, exports, label="Export (kW)")plt.plot(hours, solar_production, label="solar_production (kW)")plt.xlabel("Hour")plt.ylabel("Value")plt.title("Time Series of SOC, Import, Export, and Flexible Load")plt.legend()plt.grid(True)plt.tight_layout()plt.show()

In [ ]:
plt.plot(hours, max_FFL*np.array(load_profile_consumer), label="Desired Load Profile (kW)", marker = "*", color='red')plt.plot(hours, np.array(actual_profile), label="Actual Load Profile (kW)", linestyle='--', color='blue')plt.xlabel("Hour")plt.ylabel("Value")plt.title("Time Series of Targeted Load and Actual Flexible Load")plt.legend()plt.grid(True)plt.tight_layout()plt.show()

In [ ]:
plt.plot(hours, solar_production, label="Solar Production (kW)", marker='o')plt.plot(hours, np.array(Solar_CF)* max_power_solar, label="Max Solar Production (kW)", linestyle='--', color='gray')plt.xlabel("Hour")plt.ylabel("Power (kW)")plt.title("Time Series of Solar Production and Maximum Possible Solar Output")plt.legend()

In [ ]:
for _ in range(len(load_profile_consumer)):    print(f"Actual_load_profile / Desired_load_profile hour_{_+1}: {actual_profile[_]:.2f} / {(load_profile_consumer[_]*max_FFL):.2f} // {((actual_profile[_])/(load_profile_consumer[_]*max_FFL)):.2f} P.u")

In [ ]:
fig, ax1 = plt.subplots(figsize=(10,6))ax1.plot(hours, max_FFL*np.array(load_profile_consumer), label="Desired Load Profile (kW)", marker="*", color='red')ax1.plot(hours, np.array(actual_profile), label="Actual Load Profile (kW)", linestyle='--', color='blue')ax1.set_xlabel("Hour")ax1.set_ylabel("Load (kW)")ax1.legend(loc="upper left")ax1.grid(True)ax2 = ax1.twinx()ax2.plot(hours, penalty_discomfort, label="Penalty Discomfort", color='green', linewidth=2, alpha=0.7)ax2.set_ylabel("Penalty Discomfort")ax2.legend(loc="upper right")plt.title("Time Series of Targeted Load, Actual Flexible Load, and Penalty Discomfort")plt.tight_layout()plt.show()